### Домашняя работа №2
### Анализ дампа оперативной памяти.
### Шараев Евгений

Задание к варианту 1.

 1. Каков IP-адрес зараженного узла?
 2. Каков MAC-адрес зараженного узла?
3. Каково доменное имя зараженного узла?
4. Какие сайты посетил пользователь зараженного устройства по своему желанию?
5. Посещение каких сайтов зафиксировано в сетевом трафике?
6. Каково доменное имя сайта, с которого произошла загрузка вредоносного программного обеспечения?
7. Каков IP-адрес узла, с которого произошла загрузка вредоносного программного обеспечения?
8. Загружались ли пользователем или системой без ведома пользователя файлы, не являющиеся вредоносными?
9. Какие сайты (доменные имена) задействованы в заражении пользователя вредоносным программным обеспечением (имеют следы вредоносной активности, участвуют во вредоносных действиях)?
10. Каков механизм переходов (перенаправлений) пользователя с посещенных сайтов на сайт, с которого было загружено вредоносное программное обеспечение?


In [1]:
# Дамп оперативной памяти в файле var-1.vmem
# Также я буду использовать две версии volatility: 
# под Python2 и под Python3

!ls -l

total 1049196
drwxr-xr-x 14 evgeny evgeny       4096 Apr 11 17:06 distorm
drwxr-xr-x 14 evgeny evgeny       4096 Apr 11 17:14 distorm3
drwxr-xr-x  7 evgeny evgeny       4096 Apr  2 11:13 env
-rw-r--r--  1 evgeny evgeny     100910 Apr 12 12:38 sharaev_evgeny_hw2.ipynb
-rw-r--r--  1 evgeny evgeny     504048 Apr 12 12:18 sharaev_evgeny_hw2.pdf
-rw-r--r--  1 evgeny evgeny 1073741824 May 25  2023 var-1.vmem
drwxr-xr-x  8 evgeny evgeny       4096 Apr 11 16:10 volatility
drwxr-xr-x  8 evgeny evgeny       4096 Apr  2 10:17 volatility3


In [2]:
# Для этого у меня установлено сразу обе версии интерпретаторов

!python2.7 --version
!python3 --version

Python 2.7.13
Python 3.11.2


In [3]:
# Используемая мной версия программы volatility

!python2.7 volatility/vol.py -h | grep "Framework 2"
!python3 volatility3/vol.py -h | grep " Framework 2"

Volatility Foundation Volatility Framework 2.6.1
Volatility 3 Framework 2.7.0


In [4]:
# 1. Определите семейство и версию операционной системы на компьютере 
# Джона.

!python2.7 volatility/vol.py -f var-1.vmem imageinfo | grep -v Failed

Volatility Foundation Volatility Framework 2.6.1
INFO    : volatility.debug    : Determining profile based on KDBG search...
          Suggested Profile(s) : Win7SP1x64, Win7SP0x64, Win2008R2SP0x64, Win2008R2SP1x64_24000, Win2008R2SP1x64_23418, Win2008R2SP1x64, Win7SP1x64_24000, Win7SP1x64_23418
                     AS Layer1 : WindowsAMD64PagedMemory (Kernel AS)
                     AS Layer2 : FileAddressSpace (/home/evgeny/Projects/forensic/var-1.vmem)
                      PAE type : No PAE
                           DTB : 0x187000L
                          KDBG : 0xf80002bfd0a0L
          Number of Processors : 1
     Image Type (Service Pack) : 1
                KPCR for CPU 0 : 0xfffff80002bfed00L
             KUSER_SHARED_DATA : 0xfffff78000000000L
           Image date and time : 2020-12-27 23:06:01 UTC+0000
     Image local date and time : 2020-12-28 00:06:01 +0100


In [5]:
# Ответ: Наиболее вероятно Windows 7

In [6]:
# 2. Какой процесс на компьютере Джона установил сетевое соединение 
# с участием порта 554?

!python3 volatility3/vol.py \
    --filters LocalPort,554 \
    --file var-1.vmem \
    windows.netscan.NetScan

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
Offset	Proto	LocalAddr	LocalPort	ForeignAddr	ForeignPort	State	PID	Owner	Created

0x24d5c8c0	TCPv4	0.0.0.0	554	0.0.0.0	0	LISTENING	2368	wmpnetwk.exe	-
0x37a47480	TCPv4	0.0.0.0	554	0.0.0.0	0	LISTENING	2368	wmpnetwk.exe	-
0x37a47480	TCPv6	::	554	::	0	LISTENING	2368	wmpnetwk.exe	-


In [7]:
# Проверяем на версии 2

!python2.7 volatility/vol.py \
    --file var-1.vmem \
    --profile Win7SP1x64 \
    netscan | grep -v "Failed" | grep 554

# Offset(P)        Proto    Local Address                  Foreign Address      State            Pid      Owner          Created

Volatility Foundation Volatility Framework 2.6.1
0x24d5c8c0         TCPv4    0.0.0.0:554                    0.0.0.0:0            LISTENING        2368     wmpnetwk.exe   
0x37a47480         TCPv4    0.0.0.0:554                    0.0.0.0:0            LISTENING        2368     wmpnetwk.exe   
0x37a47480         TCPv6    :::554                         :::0                 LISTENING        2368     wmpnetwk.exe   


In [8]:
# Ответ: Процесс PID 2368

In [9]:
# 3. Устанавливались ли сетевые соединения компьютера Джона 
# с участием локальных портов в диапазоне 135-140?

!python3 volatility3/vol.py \
    --filters LocalPort,135 \
    --filters LocalPort,136 \
    --filters LocalPort,137 \
    --filters LocalPort,138 \
    --filters LocalPort,139 \
    --filters LocalPort,140 \
    --file var-1.vmem \
    windows.netscan.NetScan

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
Offset	Proto	LocalAddr	LocalPort	ForeignAddr	ForeignPort	State	PID	Owner	Created

0x3692a6d0	UDPv4	0.0.0.0	51351	*	0		940	svchost.exe	2020-12-27 23:05:56.000000 
0x3692a6d0	UDPv6	::	51351	*	0		940	svchost.exe	2020-12-27 23:05:56.000000 
0x3d6072c0	TCPv4	0.0.0.0	135	0.0.0.0	0	LISTENING	700	svchost.exe	-
0x3d6075c0	TCPv4	0.0.0.0	135	0.0.0.0	0	LISTENING	700	svchost.exe	-
0x3d6075c0	TCPv6	::	135	::	0	LISTENING	700	svchost.exe	-
0x3d721e20	UDPv4	192.168.136.131	137	*	0		4	System	2020-12-27 22:50:58.000000 
0x3d7331c0	TCPv4	192.168.136.131	139	0.0.0.0	0	LISTENING	4	System	-
0x3d736010	UDPv4	192.168.136.131	138	*	0		4	System	2020-12-27 22:50:5

In [10]:
# Проверяем на версии 2

!python2.7 volatility/vol.py \
    --file var-1.vmem \
    --profile Win7SP1x64 \
    netscan | grep -v "Failed" | grep -e "135" -e "136" -e "137" -e "138" -e "139" -e "140"

# Offset(P)          Proto    Local Address                  Foreign Address      State            Pid      Owner          Created

Volatility Foundation Volatility Framework 2.6.1
0xded3900          UDPv4    127.0.0.1:62336                *:*                                   1408     svchost.exe    2020-12-27 22:51:00 UTC+0000
0xded3ec0          UDPv6    ::1:62334                      *:*                                   1408     svchost.exe    2020-12-27 22:51:00 UTC+0000
0x217751c0         UDPv4    0.0.0.0:3702                   *:*                                   1408     svchost.exe    2020-12-27 22:52:12 UTC+0000
0x29a53010         UDPv6    ::1:1900                       *:*                                   1408     svchost.exe    2020-12-27 22:51:00 UTC+0000
0x29a53290         UDPv4    127.0.0.1:1900                 *:*                                   1408     svchost.exe    2020-12-27 22:51:00 UTC+0000
0x29a53950         UDPv4    192.168.136.131:1900           *:*                                   1408     svchost.exe    2020-12-27 22:51:00 UTC+0000
0x2a453010         UDPv6    fe80::bda1:b28:fab2:a38

In [11]:
# Ответ: да, устанавливались.

In [12]:
# 4. Если такие соединения устанавливались, укажите идентификаторы 
# процессов, устанавливавших такие соединения.

In [13]:
# Ответ: PID 4, PID 700

In [14]:
# 5. Укажите идентификатор процессов-родителей данных процессов. 

!python3 volatility3/vol.py \
    --filters PID,700 \
    --filters PID,4 \
    --file var-1.vmem \
    windows.pslist 

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

4	0	System	0xfa80024b36f0	87	550	N/A	False	2020-12-27 22:50:39.000000 	N/A	Disabled
348	340	csrss.exe	0xfa8004402b30	8	484	0	False	2020-12-27 22:50:40.000000 	N/A	Disabled
400	340	wininit.exe	0xfa8004663560	3	75	0	False	2020-12-27 22:50:41.000000 	N/A	Disabled
412	392	csrss.exe	0xfa80043bdb30	10	196	1	False	2020-12-27 22:50:41.000000 	N/A	Disabled
460	392	winlogon.exe	0xfa80046b3060	3	112	1	False	2020-12-27 22:50:41.000000 	N/A	Disabled
524	400	lsm.exe	0xfa8004868b30	9	141	0	False	2020-12-27 22:50:44.000000 	N/A	Disabled
700	508	svchost.exe

In [15]:
# Проверяем на версии 2

!python2.7 volatility/vol.py \
    --file var-1.vmem \
    --profile Win7SP1x64 \
    pslist | grep -v "Failed" | grep -e "4" -e "700"

# Offset(V)          Name                    PID   PPID   Thds     Hnds   Sess  Wow64 Start                          Exit

Volatility Foundation Volatility Framework 2.6.1
Offset(V)          Name                    PID   PPID   Thds     Hnds   Sess  Wow64 Start                          Exit                          
0xfffffa80024b36f0 System                    4      0     87      550 ------      0 2020-12-27 22:50:39 UTC+0000                                 
0xfffffa8003ad8780 smss.exe                260      4      2       29 ------      0 2020-12-27 22:50:39 UTC+0000                                 
0xfffffa8004402b30 csrss.exe               348    340      8      484      0      0 2020-12-27 22:50:40 UTC+0000                                 
0xfffffa8004663560 wininit.exe             400    340      3       75      0      0 2020-12-27 22:50:41 UTC+0000                                 
0xfffffa80043bdb30 csrss.exe               412    392     10      196      1      0 2020-12-27 22:50:41 UTC+0000                                 
0xfffffa80046b3060 winlogon.exe            460    392      3      112      

In [16]:
# Ответ
# PPID(PID 4) --> PID 0
# PPID(PID 700) --> PID 508

In [17]:
# 6. Укажите все процессы, порожденные процессами, устанавливавшими 
# выявленные в пункте 3 соединения.
# Если соединения породили PID 4 и 700 то порожденные ими процессы - 
# это их дети и в колонке PPID они будут иметь процеесы 700 и 4

!python3 volatility3/vol.py \
    --filters PPID,700 \
    --filters PPID,4 \
    --file var-1.vmem \
    windows.pslist

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

260	4	smss.exe	0xfa8003ad8780	2	29	N/A	False	2020-12-27 22:50:39.000000 	N/A	Disabled
348	340	csrss.exe	0xfa8004402b30	8	484	0	False	2020-12-27 22:50:40.000000 	N/A	Disabled
400	340	wininit.exe	0xfa8004663560	3	75	0	False	2020-12-27 22:50:41.000000 	N/A	Disabled
508	400	services.exe	0xfa8004864060	8	220	0	False	2020-12-27 22:50:43.000000 	N/A	Disabled
516	400	lsass.exe	0xfa800485e910	7	708	0	False	2020-12-27 22:50:44.000000 	N/A	Disabled
524	400	lsm.exe	0xfa8004868b30	9	141	0	False	2020-12-27 22:50:44.000000 	N/A	Disabled
1144	1124	explorer

In [18]:
# Также проверю в плагине pstree

!python3 volatility3/vol.py \
    --filters PPID,4 \
    --filters PPID,700 \
    --file var-1.vmem \
    windows.pstree

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	Audit	Cmd	Path

* 260	4	smss.exe	0xfa8003ad8780	2	29	N/A	False	2020-12-27 22:50:39.000000 	N/A	\Device\HarddiskVolume2\Windows\System32\smss.exe	\SystemRoot\System32\smss.exe	\SystemRoot\System32\smss.exe
348	340	csrss.exe	0xfa8004402b30	8	484	0	False	2020-12-27 22:50:40.000000 	N/A	\Device\HarddiskVolume2\Windows\System32\csrss.exe	%SystemRoot%\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,20480,768 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=winsrv:ConServerDllInitializ

In [19]:
# Ответ:
# PID 4 породил процесс PID 260
# PID 700 не породил ни одного процесса

In [20]:
# А какие процессы были порождены процессом породившим процесс 700? 
# То есть у каких процессов PPID 508

!python3 volatility3/vol.py \
    --filters PPID,508 \
    --file var-1.vmem \
    windows.pslist

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

612	508	svchost.exe	0xfa8004bf3610	9	350	0	False	2020-12-27 22:50:46.000000 	N/A	Disabled
676	508	vm3dservice.ex	0xfa8004bf2060	3	44	0	False	2020-12-27 22:50:48.000000 	N/A	Disabled
700	508	svchost.exe	0xfa8004bfa740	6	273	0	False	2020-12-27 22:50:48.000000 	N/A	Disabled
812	508	svchost.exe	0xfa8004c5fb30	23	576	0	False	2020-12-27 22:50:49.000000 	N/A	Disabled
852	508	svchost.exe	0xfa8004cb0390	26	530	0	False	2020-12-27 22:50:49.000000 	N/A	Disabled
896	508	svchost.exe	0xfa8004cc7b30	40	939	0	False	2020-12-27 22:50:49.000000 	N/A	Disabled
2

In [21]:
# А вот тут много всего

In [22]:
# 7. Установите, сколько различных процессов svchost.exe запускалось, 
# приведите идентификаторы запущенных процессов.

!python3 volatility3/vol.py \
    --filters ImageFileName,svchost.exe \
    --file var-1.vmem \
    windows.pslist

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	File output

612	508	svchost.exe	0xfa8004bf3610	9	350	0	False	2020-12-27 22:50:46.000000 	N/A	Disabled
700	508	svchost.exe	0xfa8004bfa740	6	273	0	False	2020-12-27 22:50:48.000000 	N/A	Disabled
812	508	svchost.exe	0xfa8004c5fb30	23	576	0	False	2020-12-27 22:50:49.000000 	N/A	Disabled
852	508	svchost.exe	0xfa8004cb0390	26	530	0	False	2020-12-27 22:50:49.000000 	N/A	Disabled
896	508	svchost.exe	0xfa8004cc7b30	40	939	0	False	2020-12-27 22:50:49.000000 	N/A	Disabled
296	508	svchost.exe	0xfa8004cfdb30	18	759	0	False	2020-12-27 22:50:50.000000 	N/A	Disabled
94

In [23]:
# Ответ:
# Всего 11 процессов с таким названием
# PID 612, 700, 812, 852, 896, 296, 940, 1248, 1408, 2632, 2008

In [24]:
# 8. Укажите имена исполняемых файлов, запустивших эти процессы.

!python3 volatility3/vol.py \
    --filters ImageFileName,svchost.exe \
    --file var-1.vmem \
    windows.pstree

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	PPID	ImageFileName	Offset(V)	Threads	Handles	SessionId	Wow64	CreateTime	ExitTime	Audit	Cmd	Path

** 896	508	svchost.exe	0xfa8004cc7b30	40	939	0	False	2020-12-27 22:50:49.000000 	N/A	\Device\HarddiskVolume2\Windows\System32\svchost.exe	C:\Windows\system32\svchost.exe -k netsvcs	C:\Windows\system32\svchost.exe
** 1408	508	svchost.exe	0xfa8004fd3b30	22	308	0	False	2020-12-27 22:50:52.000000 	N/A	\Device\HarddiskVolume2\Windows\System32\svchost.exe	C:\Windows\system32\svchost.exe -k LocalServiceAndNoImpersonation	C:\Windows\system32\svchost.exe
** 296	508	svchost.exe	0xfa8004cfdb30	18	759	0	False	2020-12-27 22:50:50.000000 	N/A	\Device\

In [25]:
# Проверю эти данные в cmdline

!python2.7 volatility/vol.py \
    --file var-1.vmem \
    --profile Win7SP1x64 \
    cmdline | grep -v "Failed" | grep svchost.exe

Volatility Foundation Volatility Framework 2.6.1
svchost.exe pid:    612
Command line : C:\Windows\system32\svchost.exe -k DcomLaunch
svchost.exe pid:    700
Command line : C:\Windows\system32\svchost.exe -k RPCSS
svchost.exe pid:    812
Command line : C:\Windows\System32\svchost.exe -k LocalServiceNetworkRestricted
svchost.exe pid:    852
Command line : C:\Windows\System32\svchost.exe -k LocalSystemNetworkRestricted
svchost.exe pid:    896
Command line : C:\Windows\system32\svchost.exe -k netsvcs
svchost.exe pid:    296
Command line : C:\Windows\system32\svchost.exe -k LocalService
svchost.exe pid:    940
Command line : C:\Windows\system32\svchost.exe -k NetworkService
svchost.exe pid:   1248
Command line : C:\Windows\system32\svchost.exe -k LocalServiceNoNetwork
svchost.exe pid:   1408
Command line : C:\Windows\system32\svchost.exe -k LocalServiceAndNoImpersonation
svchost.exe pid:   2632
Command line : C:\Windows\System32\svchost.exe -k LocalServicePeerNet
svchost.exe pid:   2008
Co

In [26]:
# Ответ:
# Все процессы svchost.exe были запущены из файла 
# C:\Windows\system32\svchost.exe

In [27]:
# 9. Среди процессов, выявленных в пункте 7, определите те, которые 
# содержат признаки заражения.

# PID 612 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 612

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [28]:
# PID 700 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 700

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [29]:
# PID 812 - Есть признаки заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 812

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm

812	svchost.exe	0x1430000	0x143ffff	VadS	PAGE_EXECUTE_READWRITE	16	1	Disabled	N/A	
41 ba 80 00 00 00 48 b8	A.....H.
38 a1 13 ff fe 07 00 00	8.......
48 ff 20 90 41 ba 81 00	H...A...
00 00 48 b8 38 a1 13 ff	..H.8...
fe 07 00 00 48 ff 20 90	....H...
41 ba 82 00 00 00 48 b8	A.....H.
38 a1 13 ff fe 07 00 00	8.......
48 ff 20 90 41 ba 83 00	H...A...	
0x1430000:	mov	r10d, 0x80
0x1430006:	movabs	rax, 0x7feff13a138
0x1430010:	jmp	qword ptr [rax]
0x1430013:	nop	
0x1430014:	mov	r10d, 0x81
0x143001a:	movabs	rax, 0x7feff13a138
0x1430024:	jmp	qw

In [30]:
# PID 852 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 852

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [31]:
# PID 896 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 896

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [32]:
# PID 296 - Есть признаки заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 296

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm

296	svchost.exe	0xeb0000	0xebffff	VadS	PAGE_EXECUTE_READWRITE	16	1	Disabled	N/A	
41 ba 80 00 00 00 48 b8	A.....H.
38 a1 13 ff fe 07 00 00	8.......
48 ff 20 90 41 ba 81 00	H...A...
00 00 48 b8 38 a1 13 ff	..H.8...
fe 07 00 00 48 ff 20 90	....H...
41 ba 82 00 00 00 48 b8	A.....H.
38 a1 13 ff fe 07 00 00	8.......
48 ff 20 90 41 ba 83 00	H...A...	
0xeb0000:	mov	r10d, 0x80
0xeb0006:	movabs	rax, 0x7feff13a138
0xeb0010:	jmp	qword ptr [rax]
0xeb0013:	nop	
0xeb0014:	mov	r10d, 0x81
0xeb001a:	movabs	rax, 0x7feff13a138
0xeb0024:	jmp	qword ptr [

In [33]:
# PID 940 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 940

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [34]:
# PID 1248 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 1248

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [35]:
# PID 1408 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 1408

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [36]:
# PID 2362 - Нет признаков заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 2632

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm



In [37]:
# PID 2008 - Есть признаки заражения

!python3 volatility3/vol.py \
    --file var-1.vmem \
    windows.malfind.Malfind \
    --pid 2008

Volatility 3 Framework 2.7.0
WARNING  volatility3.framework.layers.vmware: No metadata file found alongside VMEM file. A VMSS or VMSN file may be required to correctly process a VMEM file. These should be placed in the same directory with the same file name, e.g. var-1.vmem and var-1.vmss.
Progress:  100.00		PDB scanning finished                        
PID	Process	Start VPN	End VPN	Tag	Protection	CommitCharge	PrivateMemory	File output	Notes	Hexdump	Disasm

2008	svchost.exe	0x2600000	0x267ffff	VadS	PAGE_EXECUTE_READWRITE	128	1	Disabled	N/A	
20 00 00 00 e0 ff 07 00	........
0c 00 00 00 01 00 05 00	........
00 42 00 50 00 30 00 70	.B.P.0.p
00 60 00 00 00 00 00 00	.`......
48 8b 45 28 c7 00 00 00	H.E(....
00 00 c7 40 04 00 00 00	...@....
00 48 8b 45 28 48 8d 40	.H.E(H.@
08 48 89 c2 48 8b 45 20	.H..H.E.	
0x2600000:	and	byte ptr [rax], al
0x2600002:	add	byte ptr [rax], al
0x2600004:	loopne	0x2600005
2008	svchost.exe	0x4ea0000	0x4f9ffff	VadS	PAGE_EXECUTE_READWRITE	256	1	Disabled	N/A	
20 00 0

In [38]:
# 10. Установите, какое сообщение написал Джон в командной строке 
# в ходе сеанса, в момент которого был создан дамп.

!python2.7 volatility/vol.py \
    --file var-1.vmem \
    --profile Win7SP1x64 \
    consoles | grep -v "Failed"

Volatility Foundation Volatility Framework 2.6.1
**************************************************
ConsoleProcess: conhost.exe Pid: 2488
Console: 0xffa66200 CommandHistorySize: 50
HistoryBufferCount: 1 HistoryBufferMax: 4
OriginalTitle: %SystemRoot%\System32\cmd.exe
Title: Administrator: C:\Windows\System32\cmd.exe
AttachedProcess: cmd.exe Pid: 1920 Handle: 0x60
----
CommandHistory: 0x21e9c0 Application: cmd.exe Flags: Allocated, Reset
CommandCount: 7 LastAdded: 6 LastDisplayed: 6
FirstCommand: 0 CommandCountMax: 50
ProcessHandle: 0x60
Cmd #0 at 0x1fe3a0: cd /
Cmd #1 at 0x1f78b0: echo THM{You_found_me} > test.txt
Cmd #2 at 0x21dcf0: cls
Cmd #3 at 0x1fe3c0: cd /Users
Cmd #4 at 0x1fe3e0: cd /John
Cmd #5 at 0x21db30: dir
Cmd #6 at 0x1fe400: cd John
----
Screen 0x200f70 X:80 Y:300
Dump:
                                                                                
C:\>cd /Users                                                                   
                                           

In [39]:
# Ответ:
# Cmd #0 at 0x1fe3a0: cd /
# Cmd #1 at 0x1f78b0: echo THM{You_found_me} > test.txt
# Cmd #2 at 0x21dcf0: cls
# Cmd #3 at 0x1fe3c0: cd /Users
# Cmd #4 at 0x1fe3e0: cd /John
# Cmd #5 at 0x21db30: dir
# Cmd #6 at 0x1fe400: cd John